<a href="https://colab.research.google.com/github/sudipamandal/Chatbot-using-PyTorch/blob/master/FFNN_Corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# Load the dataset (adjust the path as necessary)
file_path = 'combined_data.xlsx'
data = pd.read_excel(file_path)

# Replace -1 with 0 for positive sentiment and 1 remains for negative sentiment
data.replace({-1: 0, 1: 1}, inplace=True)

# Determine the overall sentiment
data['overall_sentiment'] = data.drop(columns=['sentence']).apply(lambda row: 1 if row.sum() > 0 else 0, axis=1)

# Initialize tokenizer and fit on sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['sentence'])

# Convert sentences to sequences and pad them to ensure uniform length
sequences = tokenizer.texts_to_sequences(data['sentence'])
X = pad_sequences(sequences)
y = data['overall_sentiment'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
max_length = X.shape[1]  # Use the padded sequence length

model = Sequential([
    Embedding(vocab_size, 16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)


Epoch 1/10
236/236 [==============================] - 2s 6ms/step - loss: 0.4658 - accuracy: 0.8750 - val_loss: 0.3294 - val_accuracy: 0.8964
Epoch 2/10
236/236 [==============================] - 2s 9ms/step - loss: 0.3431 - accuracy: 0.8889 - val_loss: 0.3248 - val_accuracy: 0.8964
Epoch 3/10
236/236 [==============================] - 3s 15ms/step - loss: 0.3384 - accuracy: 0.8889 - val_loss: 0.3204 - val_accuracy: 0.8964
Epoch 4/10
236/236 [==============================] - 3s 13ms/step - loss: 0.3335 - accuracy: 0.8889 - val_loss: 0.3166 - val_accuracy: 0.8964
Epoch 5/10
236/236 [==============================] - 3s 13ms/step - loss: 0.3265 - accuracy: 0.8893 - val_loss: 0.3076 - val_accuracy: 0.8970
Epoch 6/10
236/236 [==============================] - 3s 11ms/step - loss: 0.3107 - accuracy: 0.8902 - val_loss: 0.2853 - val_accuracy: 0.8986
Epoch 7/10
236/236 [==============================] - 2s 10ms/step - loss: 0.2819 - accuracy: 0.8944 - val_loss: 0.2564 - val_accuracy: 0.9023
E

In [ ]:
def predict_sentiment(text):
    # Convert the sentence to a sequence
    sequence = tokenizer.texts_to_sequences([text])

    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])

    # Predict sentiment
    prediction = model.predict(padded_sequence)[0][0]

    # Interpret the prediction
    if prediction < 0.5:
        return "Positive sentiment (0)", prediction
    else:
        return "Negative sentiment (1)", prediction

# Example usage
sentence1 = "We cannot and will not be liable for any loss or damage arising from your failure to comply with the above."
sentence2 = "We may suspend or terminate your account or cease providing you with all or part of the Services"
sentence3 = "You are responsible for your use of the Services and for any Content you provide, including compliance with applicable laws, rules, and regulations."

sentiment, confidence = predict_sentiment(sentence1)
print(f"Predicted Sentiment: {sentiment}, Confidence Score: {confidence}")

sentiment, confidence = predict_sentiment(sentence2)
print(f"Predicted Sentiment: {sentiment}, Confidence Score: {confidence}")

sentiment, confidence = predict_sentiment(sentence3)
print(f"Predicted Sentiment: {sentiment}, Confidence Score: {confidence}")


1/1 [==============================] - 0s 354ms/step
Predicted Sentiment: Positive sentiment (0), Confidence Score: 0.3210330307483673
1/1 [==============================] - 0s 21ms/step
Predicted Sentiment: Positive sentiment (0), Confidence Score: 0.32103586196899414
1/1 [==============================] - 0s 26ms/step
Predicted Sentiment: Positive sentiment (0), Confidence Score: 0.032136134803295135


In [ ]:
import nltk
from keras.preprocessing.sequence import pad_sequences
nltk.download('punkt')

def print_negative_sentences(paragraph):
    # Split the paragraph into sentences
    sentences = nltk.tokenize.sent_tokenize(paragraph)

    for sentence in sentences:
        # Convert the sentence to a sequence
        sequence = tokenizer.texts_to_sequences([sentence])

        # Pad the sequence
        padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])

        # Predict sentiment
        prediction = model.predict(padded_sequence)[0][0]

        # If the prediction indicates a negative sentiment, print the sentence
        if prediction >= 0.5:  # Assuming 0.5 as the threshold for negative sentiment
            print(sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Example paragraph
paragraph = "We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable. We will make reasonable efforts to notify you by the email address associated with your account or the next time you attempt to access your account, depending on the circumstances. To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience. In all such cases, the Terms shall terminate, including, without limitation, your license to use the Services, except that the following sections shall continue to apply: 2, 3, 5, 6, and the misuse provisions of Section 4 (“Misuse of the Services”). If you believe your account was terminated in error you can file an appeal following the steps found in our Help Center (https://help.x.com/forms/account-access/appeals). For the avoidance of doubt, these Terms survive the deactivation or termination of your account."

# Call the function with the example paragraph
print_negative_sentences(paragraph)

1/1 [==============================] - 0s 21ms/step
We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable.
1/1 [==============================] - 0s 23ms/step
To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience.
1/1 [==============================] - 0s 22ms/step


In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Assuming `model` is your trained Keras model and `tokenizer` is your Keras tokenizer

# Save the model
model.save('my_model')

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
file_path = 'flipkart.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

print_negative_sentences(text)

1/1 [==============================] - 0s 21ms/step
By impliedly or expressly accepting these Terms of Use, You also accept and agree to be bound by Flipkart Policies ((including but not limited to Privacy Policy available at Privacy) as amended from time to time.
1/1 [==============================] - 0s 21ms/step
If there is reason to believe that there is likely to be a breach of security or misuse of Your account, We may request You to change the password or we may suspend Your account without any liability to Flipkart, for such period of time as we deem appropriate in the circumstances.
1/1 [==============================] - 0s 40ms/step
Flipkart shall have the right to remove or edit any content that in its sole discretion violates, or is alleged to violate, any applicable law or either the spirit or letter of these Terms of Use.
1/1 [==============================] - 0s 49ms/step
In no event shall Flipkart assume or have any responsibility or liability for any Content posted or 